In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import joblib
import nltk

from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

!pip install stop_words
import stop_words
import string
import re

In [2]:
!pip install --upgrade scikit-learn==1.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [1]:

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [6]:
raw_data = pd.read_csv('/content/TRAIN_RES_300k.csv', encoding='utf-8')
raw_data.head()

,achievements,achievements_modified,company_name,demands,id_cv,job_title
0,NaN,NaN,"ЗАО ""Тропарево""","Работа трактористом в ЗАО ""Тропарево""",17f53320-1538-11ec-8141-93a84dada081,тракторист
1,NaN,NaN,Центральный районный суд г. Челябинска,Проходил практику в судебной концелярии,8d0872f0-0f0f-11ec-91a2-3bfa22f2d66b,специалист
2,NaN,NaN,"ГБУЗ АО ""Яренская ЦРБ""",водитель скорой помощи,affd4320-0fc4-11ec-a0f5-69632329477c,водитель
3,NaN,NaN,ооо агро-инвест,погрузка овощей,7fb334a0-122a-11ec-8374-ab5d2eb93a75,грузчик
4,NaN,NaN,"БОУ г. Омска ""Средняя общеобразовательная школ...","Ведение материального стола, ведение кассы",03d28a20-0ff0-11ec-acaa-ab5d2eb93a75,бухгалтер


In [7]:
raw_data = raw_data.sample(frac=1).reset_index(drop=True)
#raw_data[:4000].to_csv('TEST_RES.csv', index=False)
raw_data = raw_data[4000:100000]

In [8]:
def preprocessing_resume_classifier(raw_data):
  # отбор полезных колонок
  raw_data = raw_data[['id_cv', 'demands', 'job_title']]


  # отбор проффессий которые указаны в более чем n_jobs резюме, для лучше точности модели
  n_jobs = 50
  raw_data = raw_data[raw_data['job_title'].isin(raw_data['job_title'].value_counts()[raw_data['job_title'].value_counts() > n_jobs].index)]
  raw_data = raw_data.reset_index(drop=True)

  # NLP препроцессинг резюме
  def remove_punctuation(text):
    return ''.join([ch if ch not in string.punctuation else ' ' for ch in text])
  def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
  def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

  prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in raw_data['demands'].astype('str')]
  raw_data['text_prep'] = prep_text

  #final = raw_data.dropna()
  return raw_data

In [9]:
def preprocessing_resume_classifier_test(raw_data):
  # отбор полезных колонок
  #raw_data = raw_data[['id_cv', 'demands', 'job_title']]
  raw_data = raw_data[['id_cv', 'demands']]

  # отбор проффессий которые указаны в более чем n_jobs резюме, для лучше точности модели
  #n_jobs = 200
  #raw_data = raw_data[raw_data['job_title'].isin(raw_data['job_title'].value_counts()[raw_data['job_title'].value_counts() > n_jobs].index)]
  #raw_data = raw_data.reset_index(drop=True)

  # NLP препроцессинг резюме
  def remove_punctuation(text):
    return ''.join([ch if ch not in string.punctuation else ' ' for ch in text])
  def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
  def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

  prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in raw_data['demands'].astype('str')]
  raw_data['text_prep'] = prep_text

  #final = raw_data.dropna()
  return raw_data

In [ ]:
data = preprocessing_resume_classifier(raw_data)
X = data ['text_prep']
y = data ['job_title']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42, test_size=0.3)
n = 100000
russian_stopwords = stop_words.get_stop_words('ru')

logreg = Pipeline([
  ('CountVectorizer', CountVectorizer(stop_words=russian_stopwords)),
  ('clf', LogisticRegression(n_jobs=-1, C=1e5, solver='saga',
                                           multi_class='multinomial',
                                           max_iter=12,
                                           random_state=43))
])

logreg.fit(X_train[:n], y_train[:n])
y_pred = logreg.predict(X_test[:n])'''

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
test = pd.read_csv('TEST_RES.csv')

test_pr = preprocessing_resume_classifier_test(test)

y_pred_t = logreg.predict(test_pr['text_prep'])
y_true = pd.read_csv('true_submission_test_res.csv')['job_title']


<ipython-input-8-6a46b2ccec4d>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['text_prep'] = prep_text


In [ ]:
print(classification_report(y_true, y_pred_t))
print(f"F1 Score: {f1_score(y_true, y_pred_t, average='micro')}")

                                                       precision    recall  f1-score   support

                                          автомеханик       0.00      0.00      0.00         2
                                          автослесарь       0.00      0.00      0.00         2
                                         автоэлектрик       0.00      0.00      0.00         1
                                   агент по снабжению       0.00      0.00      0.00         1
                                              агроном       0.00      0.00      0.00         3
                                              адвокат       0.00      0.00      0.00         2
                                        администратор       0.49      0.32      0.39       104
                                       аккумуляторщик       0.00      0.00      0.00         1
                                                актёр       0.00      0.00      0.00         1
                                             акуш

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
joblib.dump(logreg, 'resume_classifier_model_0.joblib')